In [1]:
import numpy as np
import os
import sys
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


from sklearn.metrics import confusion_matrix

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.TimeAugmentor import TimeAugmentor
from Classes.DataProcessing.NoiseAugmentor import NoiseAugmentor
from Classes.DataProcessing.RamLoader import RamLoader
from Classes.DataProcessing.RamGenerator import RamGenerator
from Classes.Modeling.InceptionTimeModel import InceptionTimeModel
from Classes.Modeling.NarrowSearchRam import NarrowSearchRam
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Modeling.ResultFitter import ResultFitter
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
import json
#from Classes import Tf_shutup
#Tf_shutup.Tf_shutup()

from livelossplot import PlotLossesKeras



from matplotlib.colors import ListedColormap

plt.rcParams["figure.figsize"]= (15,15)
helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint



1 Physical GPUs, 1 Logical GPUs


In [2]:
load_args = {
    'earth_explo_only' : False,
    'noise_earth_only' : False,
    'noise_not_noise' : True,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.4,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

if load_args['earth_explo_only']:
    full_and_noise_ds = np.concatenate((full_ds, noise_ds))
    timeAug = TimeAugmentor(handler, full_and_noise_ds, seed = load_args['seed'])
else:
    timeAug = TimeAugmentor(handler, full_ds, seed = load_args['seed'])

2 3
{'noise': 105999, 'earthquake': 105999, 'explosion': 102808}


In [3]:
############ Model picker #############
#model_nr_type = "LSTM"
#is_lstm = False
num_channels = 3

########### Hyperparameters ###########
batch_size = 512
epochs = 100
learning_rate = 0.01
opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
use_residuals = True
use_bottleneck = False
nr_modules = 1
kernel_size = 40
num_filters = 32
bottleneck_size = 26
module_output_activation = "sigmoid"
module_activation = "sigmoid"
shortcut_activation = "tanh"
output_activation = "sigmoid"
reg_module = False
reg_shortcut = True
l1_r = 0.01
l2_r = 0.01

########### Preprocessing ###########
use_noise_augmentor = True
use_time_augmentor = True
detrend = False
use_scaler = True
use_min_max = False
use_highpass = False
highpass_freq = 0.2

use_tensorboard = True
use_livelossplot = False
use_custom = False
use_reduce_lr = True


In [4]:
scaler = None
noiseAug = None
if use_time_augmentor:
    timeAug.fit()
if use_scaler:
    scaler = StandardScalerFitter(train_ds, timeAug).fit_scaler(detrend = detrend)
if use_noise_augmentor:
    noiseAug = NoiseAugmentor(train_ds, use_scaler, scaler, loadData, timeAug)

Fit process completed after 30.159095764160156 seconds. Total datapoints fitted: 84160.
Average time per datapoint: 0.0003583542747642604


In [5]:
ramLoader = RamLoader(handler, timeAug, scaler)
x_train, y_train = ramLoader.load_to_ram(train_ds, False)
x_val, y_val = ramLoader.load_to_ram(val_ds, False)

Starting loading to RAM
Completed loading to RAM
Starting loading to RAM
Completed loading to RAM


In [6]:
callbacks = []

def clear_tensorboard_dir():
    import os
    import shutil
    path = f"{base_dir}/Tensorboard_dir/fit"
    files = os.listdir(path)
    print(files)
    for f in files:
        shutil.rmtree(os.path.join(path,f))

if use_tensorboard:
    import datetime
    clear_tensorboard_dir()
    #%tensorboard --logdir tensorboard_dir/fit
    log_dir = f"{base_dir}/tensorboard_dir/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    callbacks.append(tensorboard_callback)

if use_custom:
    custom_callback = CustomCallback(data_gen)
    callbacks.append(custom_callback)
elif use_livelossplot:
    callbacks.append(PlotLossesKeras())
elif use_reduce_lr:
    callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                          min_lr=0.0001))

['20210210-160534']


In [7]:
nr_classes = len(set(y_val[:,0]))
print(nr_classes)
# Initializing model:
num_ds, channels, timesteps = handler.get_trace_shape_no_cast(train_ds, use_time_augmentor)
input_shape = (channels, timesteps)
print(input_shape)

optimizer_args = helper.generate_model_compile_args(opt, nr_classes)

build_model_args = {'input_shape' : input_shape,
                    'nr_classes' : nr_classes,
                    'optimizer' : opt,
                    'use_residuals' : use_residuals,
                    'use_bottleneck' : use_bottleneck,
                    'nr_modules' : nr_modules,
                    'kernel_size' : kernel_size,
                    'num_filters' : num_filters,
                    'bottleneck_size' : bottleneck_size,
                    'shortcut_activation'  : shortcut_activation,
                    'module_activation' : module_activation,
                    'module_output_activation' : module_output_activation,
                    'output_activation' : output_activation,
                    'reg_shortcut' : reg_shortcut,
                    'reg_module' : reg_module,
                    'l1_r' : l1_r,
                    'l2_r' : l2_r}

inceptionTime = InceptionTimeModel(**build_model_args)

2
(3, 6000)


In [8]:
gen = RamGenerator(loadData, handler)

train_gen = gen.data_generator(x_train, y_train, batch_size)
val_gen = gen.data_generator(x_val, y_val, batch_size)

args = {'steps_per_epoch' : helper.get_steps_per_epoch(train_ds, batch_size),
        'epochs' : epochs,
        'validation_data' : val_gen,
        'validation_steps' : helper.get_steps_per_epoch(val_ds, batch_size),
        'verbose' : 1,
        'use_multiprocessing' : False, 
        'workers' : 1,
        'callbacks' : callbacks
}
model = inceptionTime.build_model(input_shape, nr_classes)
model_fit = model.fit(train_gen, **args)
#model_fit = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (x_val, y_val))

val_precision_1: 0.4971 - val_binary_accuracy: 0.4970 - val_recall_1: 0.9980
Epoch 21/100
123/123 [==============================] - 4s 31ms/step - loss: 0.3773 - precision_1: 0.8818 - binary_accuracy: 0.8348 - recall_1: 0.7739 - val_loss: 1.6565 - val_precision_1: 0.4991 - val_binary_accuracy: 0.5011 - val_recall_1: 0.9828
Epoch 22/100
123/123 [==============================] - 4s 31ms/step - loss: 0.3688 - precision_1: 0.8975 - binary_accuracy: 0.8433 - recall_1: 0.7751 - val_loss: 2.5510 - val_precision_1: 0.4973 - val_binary_accuracy: 0.4974 - val_recall_1: 0.9874
Epoch 23/100
123/123 [==============================] - 4s 31ms/step - loss: 0.3690 - precision_1: 0.8941 - binary_accuracy: 0.8421 - recall_1: 0.7773 - val_loss: 1.5180 - val_precision_1: 0.9887 - val_binary_accuracy: 0.6578 - val_recall_1: 0.3154
Epoch 24/100
123/123 [==============================] - 4s 31ms/step - loss: 0.3601 - precision_1: 0.9029 - binary_accuracy: 0.8460 - recall_1: 0.7747 - val_loss: 3.6612 - val_